In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


## Begin the Manual Procedure

In [2]:
df_manual = pd.read_csv('/kaggle/input/iris/Iris.csv')
df_manual = df_manual.drop(columns=['Id'])
df_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## Manual Label Encoding

In [3]:
label = [i for i in df_manual['Species'].drop_duplicates()]
number = [i for i in range(3)]

encoding = dict(zip(label, number))
df_manual['Species'] = df_manual['Species'].replace(encoding)

/tmp/ipykernel_18/2979275018.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_manual['Species'] = df_manual['Species'].replace(encoding)


## Normalize Data Manually

In [4]:
def normalize_manual(df,
                     method='zscore'):
        
    if method == 'decimal':
        
        # Using Decimal Scaling
        for i in df.columns[:-1]:
            mini = df[i].min()
            df[i] = df[i].apply(lambda x: x / 10**mini)

    elif method == 'zscore':
        
        # Using Z-Score Scaling
        for i in df.columns[:-1]:
            std = np.std(df[i])
            mean = df[i].mean()
            df[i] = df[i].apply(lambda x: (x - mean) / std)

    elif method == 'log':
        
        # Using Logaritm Scaling
        for i in df.columns[:-1]:
            df[i] = df[i].apply(lambda x: np.log([x])[0])
    
    return df

In [5]:
df_manual = normalize_manual(df_manual)

## Split The Data into Train and Test Manually

In [6]:
def train_test_df_split(df,
                        y_column,
                        test_size,
                        randomize=0):
    
    label = [i for i in df[y_column].drop_duplicates()]
    
    # Prepare the First Data Segment based on Label
    x_train = df[df[y_column] == label[0]]
    
    # Calculate the Train Size based on the Amount of Row
    train_size = int(x_train.shape[0] * (1-test_size))
    
    # Split the first Train and Test Data Segment based on Label
    x_train = x_train.iloc[:train_size,:]
    x_test = x_train.iloc[train_size:,:]
    
    for i in label[1:]:
        
        # Prepare the next Data Segment based on Label
        temp = df[df[y_column] == i]
        
        # Calculate the next Train Size based on the Amount of Row
        train_size = int(temp.shape[0] * (1-test_size))
        
        # Split the next Train and Test Data Segment based on Label
        temp_train = temp.iloc[:train_size,:]
        temp_test = x_train.iloc[train_size:,:]
        
        # Execute the concat for Train Data
        x_train = pd.concat([x_train, temp_train],
                            ignore_index=True)
        
        # Execute the concat for Test Data
        x_test = pd.concat([x_test, temp_test],
                            ignore_index=True)
    
    # Randomize the Train Data and Reset the Index
    if randomize > 0:
        
        for i in range(randomize):
            x_train = x_train.sample(frac=1)
        
        # Reset the Index without 'index' Column
        x_train = x_train.reset_index(drop=True)
    
    # Split the Train Data's Feature and Label
    x_train, y_train = x_train.drop(columns=[y_column]), x_train[y_column]
    
    # Split the Test Data's Feature and Label
    x_test, y_test = x_test.drop(columns=[y_column]), x_test[y_column]
    
    return x_train, x_test, y_train, y_test

In [7]:
x_train, x_test, y_train, y_test = train_test_df_split(df_manual,
                                                       'Species',
                                                       0.25,
                                                       randomize=10)

## Prepare The Model (Not Manually LOL)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
# Initiate the Model
clf = KNeighborsClassifier()

# Fit the Train Feature and Train Label
clf.fit(x_train, y_train)

# Test the Model using Test Feature and Test Label
clf.score(x_test, y_test)

0.9459459459459459

## Now using Deep Learinig with Tensorflow

In [10]:
from tensorflow.keras import Sequential, layers

2024-02-09 10:41:27.922770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 10:41:27.922910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 10:41:28.106818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
model = Sequential([
    layers.Dense(32, activation='linear',
                 input_shape=[4,]),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
          epochs=100,
          validation_data=(x_test, y_test))

Epoch 1/100
4/4 [==============================] - 1s 101ms/step - loss: 0.9074 - accuracy: 0.6757 - val_loss: 0.9184 - val_accuracy: 0.7568
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.8369 - accuracy: 0.7117 - val_loss: 0.9233 - val_accuracy: 0.6216
Epoch 3/100
4/4 [==============================] - 0s 14ms/step - loss: 0.7734 - accuracy: 0.7387 - val_loss: 0.9246 - val_accuracy: 0.6486
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 0.7184 - accuracy: 0.8018 - val_loss: 0.9223 - val_accuracy: 0.5946
Epoch 5/100
4/4 [==============================] - 0s 14ms/step - loss: 0.6744 - accuracy: 0.8018 - val_loss: 0.9215 - val_accuracy: 0.5946
Epoch 6/100
4/4 [==============================] - 0s 15ms/step - loss: 0.6351 - accuracy: 0.8198 - val_loss: 0.9182 - val_accuracy: 0.5676
Epoch 7/100
4/4 [==============================] - 0s 14ms/step - loss: 0.6034 - accuracy: 0.8018 - val_loss: 0.9151 - val_accuracy: 0.4865
Epoch 8/100
4/4 [==

## Begin The Automatic Procedure

In [13]:
df_auto = pd.read_csv('/kaggle/input/iris/Iris.csv')
df_auto = df_auto.drop(columns=['Id'])
df_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## Encoding With Label Encoder

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
encoder = LabelEncoder()
df_auto['Species'] = encoder.fit_transform(df_auto['Species'])

## Normalize Data Using StandardScaler

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()
X = scaler.fit_transform(df_auto.drop(columns=['Species']))

## Train-Test Split

In [18]:
from sklearn.model_selection import train_test_split as tts

In [19]:
y = df_auto['Species']
X_train, X_test, y_train, y_test = tts(X, y,
                                       random_state=20,
                                       test_size=0.25)

## Model Import and Training

In [20]:
# Initiate the Model
clf = KNeighborsClassifier()

# Fit the Train Feature and Train Label
clf.fit(X_train, y_train)

# Test the Model using Test Feature and Test Label
clf.score(X_test, y_test)

0.8947368421052632

## Now using Deep Learining with Tensorflow

In [21]:
model = Sequential([
    layers.Dense(32, activation='linear',
                 input_shape=[4,]),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train,
          epochs=100,
          validation_data=(X_test, y_test))

Epoch 1/100
4/4 [==============================] - 1s 71ms/step - loss: 0.6427 - accuracy: 0.7232 - val_loss: 0.6742 - val_accuracy: 0.7105
Epoch 2/100
4/4 [==============================] - 0s 14ms/step - loss: 0.6060 - accuracy: 0.7679 - val_loss: 0.6452 - val_accuracy: 0.7105
Epoch 3/100
4/4 [==============================] - 0s 13ms/step - loss: 0.5729 - accuracy: 0.7946 - val_loss: 0.6192 - val_accuracy: 0.7368
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.5441 - accuracy: 0.8036 - val_loss: 0.5960 - val_accuracy: 0.7368
Epoch 5/100
4/4 [==============================] - 0s 13ms/step - loss: 0.5180 - accuracy: 0.8304 - val_loss: 0.5750 - val_accuracy: 0.7632
Epoch 6/100
4/4 [==============================] - 0s 13ms/step - loss: 0.4948 - accuracy: 0.8482 - val_loss: 0.5562 - val_accuracy: 0.7368
Epoch 7/100
4/4 [==============================] - 0s 13ms/step - loss: 0.4734 - accuracy: 0.8571 - val_loss: 0.5394 - val_accuracy: 0.7895
Epoch 8/100
4/4 [===